In [1]:
!export FORCE_CMAKE="1" 
!export LLAMA_CUBLAS="1"
!export CMAKE_ARGS="-DLLAMA_CUBLAS=on"

In [ ]:
from huggingface_hub import hf_hub_download

hf_hub_download(repo_id="mmnga/aixsatoshi-Swallow-MX-8x7b-NVE-chatvector-Mixtral-instruct-v2-gguf", filename="aixsatoshi-Swallow-MX-8x7b-NVE-chatvector-Mixtral-instruct-v2-q6_K.gguf", local_dir="/notebooks/model_cards")

(…)chatvector-Mixtral-instruct-v2-q6_K.gguf:   0%|          | 0.00/38.4G [00:00<?, ?B/s]

In [ ]:
!pip install llama-cpp-python --prefer-binary --upgrade --force-reinstall --no-cache-dir --extra-index-url=https://jllllll.github.io/llama-cpp-python-cuBLAS-wheels/AVX2/cu120

In [1]:
from datasets import load_dataset

dataset = load_dataset('hpprc/jawiki-news', split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3424 [00:00<?, ? examples/s]

In [2]:
from llama_cpp import Llama

# LLMの準備
llm = Llama(model_path="/notebooks/model_cards/aixsatoshi-Swallow-MX-8x7b-NVE-chatvector-Mixtral-instruct-v2-q6_K.gguf", n_ctx=8096, n_threads=1, n_gpu_layers=-1)

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   yes
ggml_init_cublas: CUDA_USE_TENSOR_CORES: no
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA RTX A6000, compute capability 8.6, VMM: yes
llama_model_loader: loaded meta data with 24 key-value pairs and 995 tensors from /notebooks/model_cards/aixsatoshi-Swallow-MX-8x7b-NVE-chatvector-Mixtral-instruct-v2-q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 32000
llama_model_loader: - kv   3:                       llama.context_length u32              = 32768
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: -

In [3]:
users = {
    0: "小学生です。どんなことに対しても「なんでそうなるのか」が気になります。",
    1: "この分野を専攻している大学生です。専門的な知識を身に着けようとしています。",
    2: "社会人です。実務として使える知識・技術を欲しています。",
    3: "この分野を専門としている大学教授です。自身の研究に役立つ知識を必要としています。",
    4: "数学者です。どんなことに対しても、物事の原理・原則が気になります。",
    5: "気分屋です。突然興味を持つ対象が変わることがあります。",
    6: "意地悪な人です。AIアシスタントに対して間違った質問をします。",
    7: "データサイエンティストです。データ分析のために必要な情報を整理しようとしています。",
}

In [4]:
import random

In [5]:
def generate_prompt(title, text, user):
    user_prompt = f'''<TITLE>
{title}
</TITLE>
<TEXT>
{text}
</TEXT>

上記のテキストを参考に、AIアシスタントへ問いかけを行うUSERと返答するASSISTANTの会話を作成してください。
USERは{user}
USERはまず指示の前提となる情報を提供します。
USERは自分が満足するまでASSISTANTに対して指示を行います。
USERはASSISTANTの応答に対して返答を行います。
ASSISTANTは優秀なAIアシスタントで、USERからの指示に応えようとします。
ASSISTANTは「です・ます」調で応答します。
ASSISTANTはUSERからの指示に対して、聞かれたこと以外にもUSERが欲していそうな情報を付け加えようとします。
<FORMAT>
USER: 
ASSISTANT: 
USER: 
ASSISTANT: 
USER: 
ASSISTANT: 
</FORMAT>'''

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = "{b_inst} {system}{prompt} {e_inst} USER: ".format(
        b_inst=B_INST,
        system=f"{B_SYS}あなたは優秀な日本人のAIアシスタントです。ユーザーからの指示に対して正確に回答してください。回答はMarkdown形式で行ってください。{E_SYS}",
        prompt=user_prompt,
        e_inst=E_INST,
    )
    return prompt


In [6]:
def pred(prompt): # 推論の実行
    output = llm(
        prompt,
        temperature=0.8,
        max_tokens=8096,
        echo=True,
    )
    return output["choices"][0]["text"][len(prompt):]

In [7]:
#datasetの30000～40000までをdataframeに変換
import pandas as pd
import json
import random

output_file_path = "wikinews-multi-turn-v1.jsonl"

In [ ]:
k = 0

while True:
    print(f"Processing {k}...")
    user_id = random.randint(0, len(users)-1)
    user = users[user_id]
    text_id = random.randint(0,len(dataset)-1)
    data = dataset[text_id]
    text = data['text']
    if len(text) > 5000:
        text = text[:5000]
    output = pred(generate_prompt(data['title'], text, user))
    print(output)
    result = {"chat": output, "text_id": data['id'], "text_title": data['title'], "text": text, "user": user}
    with open(output_file_path, "a", encoding="utf-8") as outfile:
        json_line = json.dumps(result, ensure_ascii=False)
        outfile.write(json_line + "\n")
    k += 1

Processing 0...
鳥インフルエンザの感染が広がっているようですね。このウイルスはどのように人に感染するのでしょうか？
ASSISTANT: 鳥インフルエンザウイルスは、通常、鶏やアヒルなどの鳥類に感染します。人への感染は、感染した鳥との接触や、感染した鳥の排せつ物などに触れることで起こると考えられています。しかし、人から人への感染は非常にまれであり、主に感染した家禽（鶏やアヒルなど）を処理する作業者に限られるとされています。
USER: なるほど、鳥から人への感染が一般的だと。では、なぜインドネシア政府はワクチン播種を選んだのでしょう？
ASSISTANT: インドネシア政府がワクチン播種を選んだ理由は、養鶏場経営者への経済補償の問題にあるとされています。大規模なニワトリの処分を行うと、養鶏場経営者に多額の補償が必要となります。そのため、インドネシア政府はワクチン播種を選んだと考えられます。しかし、ワクチン播種は感染を完全に防げるわけではありません。鳥インフルエンザウイルスが変異し、人への感染力が高まる可能性があるため、世界保健機関(WHO)は引き続き警戒を呼びかけています。
USER: ワクチン播種によって、ウイルスが変異する可能性があるのですか？
ASSISTANT: はい、その可能性があります。ワクチンを接種した鶏の体内で、ウイルスが変異することがあります。変異したウイルスが人に感染しやすくなったり、重症化させたりする可能性があるため、ワクチン播種は慎重に行う必要があります。世界保健機関(WHO)では、鳥インフルエンザの感染拡大を防ぐために、養鶏場でのニワトリの大規模な処分を勧告しています。
USER: ワクチン播種とウイルスの変異、それぞれのメリットとデメリットを教えてください。
ASSISTANT: ワクチン播種のメリットは、鶏の大規模な処分を行う必要がなく、経済的負担を軽減できる点にあります。一方、デメリットはウイルスの変異リスクです。ワクチン接種後も感染しないわけではなく、感染した鶏から排せつされるウイルスが変異する可能性があります。
大規模な処分のメリットは、ウイルスを根絶できる点にあります。デメリットは、養鶏場経営者への補償問題や、食料供給への影響などが挙げられます。
USER: ワクチン播種と大規模処分、それぞれについて最悪のシナリ


llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     310.92 ms /  1309 runs   (    0.24 ms per token,  4210.07 tokens per second)
llama_print_timings: prompt eval time =    6615.76 ms /   937 tokens (    7.06 ms per token,   141.63 tokens per second)
llama_print_timings:        eval time =  115887.63 ms /  1308 runs   (   88.60 ms per token,    11.29 tokens per second)
llama_print_timings:       total time =  127114.71 ms
Llama.generate: prefix-match hit


この試合で、駒大苫小牧はどのようにして4点を獲得したのですか？
ASSISTANT: 1回裏に、本間選手の三塁打と相手の失策により2点を先制しました。その後、山口選手の2点適時打でさらに2点を加え、4点を獲得したのです。
USER: その後、智辯和歌山は1点を返していますが、どのようにして1点を獲得したのでしょう？
ASSISTANT: 2回表、古宮選手の2点適時打で1点を返しました。この試合では両チームともに走者を出す場面が多く、得点機会が何度かあったのですが、その中の一つです。
USER: 駒大苫小牧が3回表に2点追加しましたが、どのようにしてこの2点を獲得したのでしょう？
ASSISTANT: 3回裏に山口選手がタイムリー二塁打を放ち、2点を追加しました。これにより、駒大苫小牧は試合を有利に進めることができたのです。
USER: なぜ8回表に早実が1点追加できたのですか？ 
ASSISTANT: 8回裏、斎藤選手が適時二塁安打を放ち、1点を獲得しました。これにより、早実は試合の流れをつかみ、完封勝ちをおさめることができたのです。
Processing 2...



llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     127.62 ms /   486 runs   (    0.26 ms per token,  3808.21 tokens per second)
llama_print_timings: prompt eval time =   11519.95 ms /  1376 tokens (    8.37 ms per token,   119.44 tokens per second)
llama_print_timings:        eval time =   48838.37 ms /   485 runs   (  100.70 ms per token,     9.93 tokens per second)
llama_print_timings:       total time =   61722.56 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     173.05 ms /   692 runs   (    0.25 ms per token,  3998.77 tokens per second)
llama_print_timings: prompt eval time =    6373.37 ms /   880 tokens (    7.24 ms per token,   138.07 tokens per second)
llama_print_timings:        eval time =   55665.05 ms /   691 runs   (   80.56 ms per token,    12.41 tokens per second)
llama_print_timings:       total time =   63982.49 ms
Llama.generate: prefix-

福本潤一参議院議員が離党したことで、7月の参院選への出馬はどうなるのでしょうか？
ASSISTANT: 福本議員は、7月の参院選に野党から出馬を希望しています。ただ、民主党幹部は集票が期待できず、党内外からの反発が予想されることから、福本議員を擁立する意向は低いようです。
<br>
USER: そもそもなぜ福本議員は離党に至ったのでしょう？
ASSISTANT: 福本議員が離党を決意した理由は、今年7月の参院選で公明党から公認されなかったことに対する不満が背景にあると考えられます。読売新聞によると、福本議員は1995年に新進党から出馬し、初当選。その後、新進党の分裂で公明党に移り、現在2期目です。
<br>
USER: 福本議員の離党が与党間の参院選対策にどのような影響をもたらすのでしょうか？
ASSISTANT: 福本議員の離党は、与党間の参院選対策に波紋を広げる可能性があります。通常、与党議員が単独で離党して野党から出馬することは異例であり、政界全体においても大きな注目を浴びています。
<br>
USER: 公明党は福本議員の離党届を受理しないと発表しましたが、除名処分の背景について教えてください。
ASSISTANT: 公明党の北側一雄幹事長は、福本議員の離党届を受理せず、除名処分を下す方針を示しました。これは、支持者への裏切り行為とみなされているためです。福本議員が公明党に所属していた間、どのような功績や影響をもたらしたかについては、具体的な情報がないため、コメントできません。
<br>
Processing 3...
河合隼雄さんってどんな人？
ASSISTANT: 河合隼雄さんは、臨床心理学者であり、文化庁長官や京都大学名誉教授を務めた方です。ユング派分析家としても知られています。
USER: どうして有名になったの？
ASSISTANT: 河合隼雄さんは、1982年に大佛次郎賞、1988年に新潮社学芸賞を受賞されました。その後、2000年に文化功労者に選出されています。これらの受賞や文化庁長官を務めた経験が、その名を高める一因となりました。
USER: どうして文化庁長官になったの？
ASSISTANT: 河合隼雄さんは京都大学や国際日本文化研究センターで教授や所長を歴任されました。これらの経験により、豊富な知識とリーダーシップが認められ、2002年に

...


llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     313.49 ms /  1247 runs   (    0.25 ms per token,  3977.84 tokens per second)
llama_print_timings: prompt eval time =    5481.73 ms /   647 tokens (    8.47 ms per token,   118.03 tokens per second)
llama_print_timings:        eval time =  103158.76 ms /  1246 runs   (   82.79 ms per token,    12.08 tokens per second)
llama_print_timings:       total time =  112768.39 ms
Llama.generate: prefix-match hit


榎本喜八ってどんな選手だったの？
ASSISTANT: 榎本喜八氏は、元プロ野球選手で「安打製造機」や「ミスターオリオンズ」と呼ばれていました。東京都出身で、早稲田実業学校高等部在籍時代の1954年に甲子園大会に春・夏連続出場を果たされています。その後、毎日オリオンズ（現:千葉ロッテマリーンズ）に入団し、新人王や2度の首位打者など数多くのタイトルを獲得されています。通算で2314安打・打率2割9分8厘・246本塁打という素晴らしい成績を残されています。
USER: そんなに活躍してたのに、引退後は野球界と距離を置いていたの？
ASSISTANT: はい、榎本喜八氏は現役引退後、あまり野球界と関わらなくなったようです。名球会の会合にも出席されることはなかったそうです。打撃技術を徹底的に追求する性格だったと言われており、現役末期に座禅を組むなど独自の取り組みもされていました。
USER: オリオンズで活躍した選手は他にもいる？
ASSISTANT: 毎日・大毎・東京・ロッテと変化したオリオンズには、榎本喜八氏のほかにも多くの名選手が在籍されていました。例えば、「ミサイル打線」でおなじみの山内一弘氏や田宮謙次郎氏もオリオンズで活躍されていました。榎本喜八氏はそうしたスター選手たちと共に、「ミスターオリオンズ」と呼ばれる存在であったようです。
<BR>
USER: 安打製造機じゃなくて、ホームランバッターだったんじゃないの？
ASSISTANT: 榎本喜八氏は確かに通算246本塁打という長距離打者としても優れた成績を残されています。しかし、安打製造機と呼ばれる由縁はその高い打率にあります。榎本氏の通算打率は2割9分8厘であり、これはプロ野球史上15位という驚くべき成績です。安打を量産したことから、「安打製造機」と呼ばれるようになったのです。
Processing 10...



llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     207.56 ms /   818 runs   (    0.25 ms per token,  3941.09 tokens per second)
llama_print_timings: prompt eval time =    8522.08 ms /  1066 tokens (    7.99 ms per token,   125.09 tokens per second)
llama_print_timings:        eval time =   74677.07 ms /   817 runs   (   91.40 ms per token,    10.94 tokens per second)
llama_print_timings:       total time =   85711.52 ms
Llama.generate: prefix-match hit


福岡6区補選で自民党はどんな動きを見せているの？
ASSISTANT: 福岡6区補選では、自民党福岡県連が公認申請している蔵内謙氏と、古屋圭司選対委員長が会談を行なったようです。その中で、鳩山邦夫元法務大臣の長男であり、既に出馬表明している鳩山二郎大川市長がリードしているという世論調査の結果を踏まえ、蔵内氏に対して出馬辞退を促しました。ただ、蔵内氏は立候補の意思を変えなかったと報じられています。自民党執行部では候補者を一本化したい考えがあるようですが、現在は2名の方が出馬の準備を進めている状況です。
USER: 鳩山二郎さんってどんな人？
ASSISTANT: 鳩山二郎大川市長は、鳩山邦夫元法務大臣の長男です。2014年に大川市長選挙に当選し、現在2期目を務めていらっしゃいます。父である鳩山邦夫元法務大臣は、自民党の重要ポストを歴任された政治家でした。鳩山二郎市長は、地域活性化に熱心に取り組んでおり、農業や文化、スポーツなど様々な分野で大川市の発展に貢献しています。
USER: 他の政党はどうなってるの？
ASSISTANT: 民進党は元在インド日本総領事館職員の新井富美子氏を擁立する予定です。共産党は筑後地区委員長の小林解子氏を支持しています。さらに、野党共闘の動きもあると報じられています。各政党とも、選挙戦でそれぞれの主張や政策をアピールし、有権者の支持を獲得しようとしている状況です。
USER: なるほど、ありがとう。
ASSISTANT: 選挙は、私たち国民に政治へ参加する機会でもあります。候補者や各政党の公約をしっかりと確認し、投票に向けて情報収集を行うことが大切です。ご不明な点などがあれば、いつでもお答えいたします。お気軽にお声がけください。
Processing 11...



llama_print_timings:        load time =    3219.00 ms
llama_print_timings:      sample time =     217.58 ms /   807 runs   (    0.27 ms per token,  3709.07 tokens per second)
llama_print_timings: prompt eval time =    5671.88 ms /   798 tokens (    7.11 ms per token,   140.69 tokens per second)
llama_print_timings:        eval time =   66855.54 ms /   806 runs   (   82.95 ms per token,    12.06 tokens per second)
llama_print_timings:       total time =   75134.96 ms
Llama.generate: prefix-match hit
